In [21]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Read the file


In [4]:
from google.colab import files

uploaded = files.upload()


Saving cancer.csv to cancer.csv


In [81]:
dataset = pd.read_csv("cancer.csv")

In [7]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  plt.figure()
  plt.title('Loss vs Iterations')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(hist['epoch'], hist['loss'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()


In [153]:
x = np.asanyarray(dataset.drop(columns=['Class']))
y = np.asanyarray(dataset[['Class']])
x = StandardScaler().fit_transform(x)
print(x.shape)
print(y.shape)

(683, 9)
(683, 1)


In [154]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2)


In [141]:
!pip install -U keras-tuner
import kerastuner as kt

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [158]:
def model_builder(hp):

  model = keras.Sequential()

  # Create Hyperparameters Space
  hp_activation=hp.Choice('dense_activation',
                       values=['relu', 'tanh', 'sigmoid'])
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
 
  # Neural Layer
  model.add(keras.layers.Dense(units = hp_units, 
                               activation = hp_activation,
                               input_shape=[x.shape[1]]))

  # Output layer
#  model.add(keras.layers.Dense(3, activation='softmax'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  # Create Hyperparameter Space
  hp_learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2,
                      sampling='LOG',  default=1e-3)
  
  # Compile model
  model.compile(loss='binary_crossentropy',
                optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate),
                metrics=['accuracy'])
  return  model

In [159]:
tuner = kt.BayesianOptimization(model_builder, 
                                objective = 'val_accuracy', 
                                max_trials = 30,
                                directory = 'Results',
                                project_name = 'Bayesian_Uno')

INFO:tensorflow:Reloading Oracle from existing project Results/Bayesian_Uno/oracle.json


In [156]:
import IPython
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [160]:
tuner.search(xtrain, ytrain, epochs=20, validation_split=0.15,
             callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print('Complete')

INFO:tensorflow:Oracle triggered exit
Complete


In [161]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(xtrain, ytrain, epochs = 50, validation_data = (xtest, ytest))

Epoch 1/50
18/18 [==============================] - 0s 8ms/step - loss: 0.5266 - accuracy: 0.6777 - val_loss: 0.3432 - val_accuracy: 0.9197
Epoch 2/50
18/18 [==============================] - 0s 4ms/step - loss: 0.2685 - accuracy: 0.9396 - val_loss: 0.2007 - val_accuracy: 0.9562
Epoch 3/50
18/18 [==============================] - 0s 4ms/step - loss: 0.1706 - accuracy: 0.9707 - val_loss: 0.1429 - val_accuracy: 0.9416
Epoch 4/50
18/18 [==============================] - 0s 4ms/step - loss: 0.1295 - accuracy: 0.9652 - val_loss: 0.1170 - val_accuracy: 0.9562
Epoch 5/50
18/18 [==============================] - 0s 4ms/step - loss: 0.1093 - accuracy: 0.9689 - val_loss: 0.1056 - val_accuracy: 0.9562
Epoch 6/50
18/18 [==============================] - 0s 4ms/step - loss: 0.0988 - accuracy: 0.9725 - val_loss: 0.0958 - val_accuracy: 0.9562
Epoch 7/50
18/18 [==============================] - 0s 4ms/step - loss: 0.0926 - accuracy: 0.9744 - val_loss: 0.0924 - val_accuracy: 0.9562
Epoch 8/50
18/18 [==

In [131]:
!rm -rf Results/